## Check version

In [1]:
import sqlalchemy
print (sqlalchemy.__version__)

1.1.5


## Connect

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
print engine.execute("select 1").scalar()

2017-05-04 10:19:52,048 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-05-04 10:19:52,051 INFO sqlalchemy.engine.base.Engine ()
2017-05-04 10:19:52,053 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-05-04 10:19:52,055 INFO sqlalchemy.engine.base.Engine ()
2017-05-04 10:19:52,059 INFO sqlalchemy.engine.base.Engine select 1
2017-05-04 10:19:52,061 INFO sqlalchemy.engine.base.Engine ()
1


## Declare mapping

In [4]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

## Make mapping class

In [6]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users' # meta date
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    def __init__(self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password
    def __repr__(self):
        return "<User('%s', '%s', '%s')>" % (self.name, self.fullname, self.password)

In [7]:
print User.__tablename__

users


In [8]:
print User.__mapper__

Mapper|User|users


## Create table using metadata

In [9]:
Base.metadata.create_all(engine)

2017-05-04 10:20:46,066 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-05-04 10:20:46,068 INFO sqlalchemy.engine.base.Engine ()
2017-05-04 10:20:46,071 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-05-04 10:20:46,072 INFO sqlalchemy.engine.base.Engine ()
2017-05-04 10:20:46,073 INFO sqlalchemy.engine.base.Engine COMMIT


## 완전상세돋는 묘사

In [5]:
from sqlalchemy import Sequence
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    password = Column(String(12))
    def __init__ (self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password
    def __repr(self):
        return "User('%s', '%s', '%s')>" % (self.name, self.fullname, self.password)

NameError: name 'Column' is not defined

## Make instance from mapping class

In [10]:
ed_user = User('haruair', 'Edward Kim', '1234')
print (ed_user.name)
print (ed_user.password)
str(ed_user.id)

haruair
1234


'None'

## Make session

In [11]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [12]:
Session = sessionmaker()

In [13]:
Session.configure(bind=engine)

In [15]:
session = Session()

## Patterns for creating session

In [16]:
ed_user = User('haruair', 'Edward Kim', '1234')
session.add(ed_user)

In [17]:
our_user = session.query(User).filter_by(name='haruair').first()

2017-05-04 10:27:21,079 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-05-04 10:27:21,082 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-05-04 10:27:21,084 INFO sqlalchemy.engine.base.Engine ('haruair', 'Edward Kim', '1234')
2017-05-04 10:27:21,086 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-05-04 10:27:21,088 INFO sqlalchemy.engine.base.Engine ('haruair', 1, 0)


In [18]:
our_user

<User('haruair', 'Edward Kim', '1234')>

In [19]:
ed_user is our_user

True

In [21]:
session.add_all([
    User('wendy', 'Wendy Willianms', 'foobar'),
    User('mary', 'Mary Contray', 'xxg527'),
    User('fred', 'Fred Flinstone', 'blar')
])

In [22]:
session.new

IdentitySet([<User('mary', 'Mary Contray', 'xxg527')>, <User('wendy', 'Wendy Willianms', 'foobar')>, <User('fred', 'Fred Flinstone', 'blar')>])

In [23]:
session.commit()

2017-05-04 10:29:25,865 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-05-04 10:29:25,868 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Willianms', 'foobar')
2017-05-04 10:29:25,871 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-05-04 10:29:25,872 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contray', 'xxg527')
2017-05-04 10:29:25,875 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-05-04 10:29:25,877 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blar')
2017-05-04 10:29:25,880 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
ed_user.id

2017-05-04 10:29:44,551 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-05-04 10:29:44,553 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2017-05-04 10:29:44,555 INFO sqlalchemy.engine.base.Engine (1,)


1

## Send query

In [27]:
for instance in session.query(User).order_by(User.id):
    print instance.name, instance.fullname

2017-05-04 10:31:53,940 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2017-05-04 10:31:53,940 INFO sqlalchemy.engine.base.Engine ()
haruair Edward Kim
wendy Wendy Willianms
mary Mary Contray
fred Fred Flinstone


In [29]:
for row in session.query(User, User.name).all():
    print row.User, row.name

2017-05-04 10:32:33,983 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-05-04 10:32:33,986 INFO sqlalchemy.engine.base.Engine ()
<User('haruair', 'Edward Kim', '1234')> haruair
<User('wendy', 'Wendy Willianms', 'foobar')> wendy
<User('mary', 'Mary Contray', 'xxg527')> mary
<User('fred', 'Fred Flinstone', 'blar')> fred


In [30]:
for row in session.query(User.name.label('name_label')).all():
    print row.name_label

2017-05-04 10:33:09,654 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2017-05-04 10:33:09,655 INFO sqlalchemy.engine.base.Engine ()
haruair
wendy
mary
fred


LIMIT 이나 OFFSET을 포함한 기본적인 Query 동작은 order by와 함께 파이썬 배열에서 쪼개는 방식

In [31]:
for user in session.query(User).order_by(User.id)[1:3]:
    print user

2017-05-04 10:34:30,680 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2017-05-04 10:34:30,681 INFO sqlalchemy.engine.base.Engine (2, 1)
<User('wendy', 'Wendy Willianms', 'foobar')>
<User('mary', 'Mary Contray', 'xxg527')>


결과물을 filter할때에는 filter_by()

In [32]:
for name in session.query(User.name).filter_by(fullname='Edward Kim'):
    print name

2017-05-04 10:42:17,548 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-05-04 10:42:17,549 INFO sqlalchemy.engine.base.Engine ('Edward Kim',)
(u'haruair',)


In [33]:
for name in session.query(User.name).filter(User.fullname=='Edward Kim'):
    print name

2017-05-04 10:43:00,358 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-05-04 10:43:00,361 INFO sqlalchemy.engine.base.Engine ('Edward Kim',)
(u'haruair',)


일반 필터 연산자들

query.filter(User.name == 'ed')

query.filter(User.name != 'ed')

query.filter(User.name.like('%ed%'))

query.filter(User.name.in_(['ed', 'wendy', 'jack']))